In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.1 MB/s eta 0:00:00


In [3]:
!pip install tashaphyne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.5/251.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 15.2 MB/s eta 0:00:00


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os
import PyPDF2
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from tashaphyne.stemming import ArabicLightStemmer

import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
# المسار إلى المجلد الذي تحتوي عليه ملفات الإجابات (في هذا المثال D:/Answers)
folder_path = '/content/drive/MyDrive/cos_sim'


In [7]:
#  ذكر اسم الملف مع نسبة التشابة

# إعداد المعالج المناسب لتحليل النص
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
als = ArabicLightStemmer()

# قائمة الإجابات وأسماء الملفات
answers_list = []
file_names = []

# قراءة جميع ملفات الإجابات في المجلد وإضافتها إلى قائمة الإجابات وأسماء الملفات
for filename in os.listdir(folder_path):
    if filename.endswith('.pdf'):
        pdf_file = open(os.path.join(folder_path, filename), 'rb')
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        # استخراج النص الخام من الصفحات في الملف الحالي
        text = ''
        for page in pdf_reader.pages:
            text += page.extract_text()
        # تجهيز النص لتحليله
        if 'ar' in filename.lower():
            # تحليل النص باللغة العربية
            text = als.light_stem(text)
        else:
            # تحليل النص باللغة الإنجليزية
            tokens = word_tokenize(text)
            # إزالة الكلمات الوظيفية
            filtered_tokens = [w for w in tokens if not w in stop_words]
            # تطبيق الجذر الأساسي على الكلمات
            stemmed_tokens = [ps.stem(w) for w in filtered_tokens]
            text = ' '.join(stemmed_tokens)
        # إضافة النص المعالج إلى قائمة الإجابات واسم الملف إلى قائمة أسماء الملفات
        answers_list.append(text)
        file_names.append(filename)

# حساب درجات التشابه بين جميع الإجابات باستخدام مقياس التشابه Cosine Similarity
similarities = []
for i in range(len(answers_list)):
    for j in range(i+1, len(answers_list)):
        answer1 = answers_list[i]
        answer2 = answers_list[j]
        # تحويل النص إلى قطع من المصفوفات لحساب Cosine Similarity
        answer1_tokens = answer1.split()
        answer2_tokens = answer2.split()
        all_tokens = set(answer1_tokens + answer2_tokens)
        answer1_vector = [answer1_tokens.count(token) for token in all_tokens]
        answer2_vector = [answer2_tokens.count(token) for token in all_tokens]
        # حساب Cosine Similarity بين النصين
        dot_product = sum([answer1_vector[k] * answer2_vector[k] for k in range(len(all_tokens))])
        norm_answer1 = sum([x**2 for x in answer1_vector]) ** 0.5
        norm_answer2 = sum([x**2 for x in answer2_vector]) ** 0.5
        similarity = dot_product / (norm_answer1 * norm_answer2)
        # إضافة النتيجة إلى قائمة النتائج
        similarities.append((file_names[i] + ' and ' + file_names[j], similarity))

# طباعة قائمة بأسماء الملفات ونسبة التشابه المحسوبة لكل منها
for pair in similarities:
    print(pair[0] + ' : ' + str(pair[1]))

Assignment_1.pdf and Assignment_2.pdf : 0.32292989923754106
Assignment_1.pdf and Assignment_3.pdf : 0.32292989923754106
Assignment_1.pdf and Assignment_4.pdf : 0.9999999999999998
Assignment_1.pdf and Assignment_5.pdf : 0.9789922711782547
Assignment_2.pdf and Assignment_3.pdf : 1.0
Assignment_2.pdf and Assignment_4.pdf : 0.32292989923754106
Assignment_2.pdf and Assignment_5.pdf : 0.3200072720569957
Assignment_3.pdf and Assignment_4.pdf : 0.32292989923754106
Assignment_3.pdf and Assignment_5.pdf : 0.3200072720569957
Assignment_4.pdf and Assignment_5.pdf : 0.9789922711782547
